# Deep Learning Project: Linear Regression with PyTorch

## Project Overview

This project guides you through the process of building and training a linear regression model using PyTorch. We will cover the following steps:

1.  **Data Loading and Preparation:** Load the advertising dataset and split it into training and testing sets.
2.  **Model Definition:**  Create a custom PyTorch model for linear regression.
3.  **Model Training:** Train the model using gradient descent.
4.  **Visualization:**  Visualize the data, loss function, and the trained linear model.

**Dataset:**  Advertising dataset (advertising.csv) containing TV, Radio, and Newspaper advertisement spending, and the resulting Sales.

## Step 1: Data Loading and Preparation

We'll begin by loading the advertising dataset, which contains information about advertisement spending on TV, radio, and newspapers, along with the corresponding sales figures. Our goal is to predict sales based on advertisement spending, and we'll assume a linear relationship between these variables.

The dataset is in a CSV file named `advertising.csv`.  We'll use the pandas library to load it into a DataFrame.  Then, we will convert the relevant columns into PyTorch tensors and split the data into training (80%) and testing (20%) sets. Finally we plot the data.

In [1]:
import pandas as pd

# Read the advertising.csv file from the same folder as the .ipynb file
df = pd.read_csv('advertising.csv')

# Display the first few rows of the dataframe to verify
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,12.0
3,151.5,41.3,58.5,16.5
4,180.8,10.8,58.4,17.9


In [ ]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import numpy as np
import pandas as pd

adv_dataset=pd.read_csv('./advertising.csv')
adv_dataset.head()

In [ ]:
# Extract feature variables and target variable
features = adv_dataset[['TV', 'Radio', 'Newspaper']].values
target = adv_dataset['Sales'].values

# Convert to PyTorch Tensors
x = torch.tensor(features, dtype=torch.float32)
y = torch.tensor(target, dtype=torch.float32)

print(x.size(),y.size())

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=50)

print(x_train.size(),y_train.size())
print(x_test.size(),y_test.size())

# Plot the first feature 'TV' in relationship with the target 'y'
plt.figure(figsize=(10, 6))
# Plot training data
plt.scatter(x_train[:, 0], y_train, color='green', label='Train data(%80)', s=5)
# Plot testing data
plt.scatter(x_test[:, 0], y_test, color='red', label='Test data(%20)', s=20)
plt.xlabel('TV Ads')
plt.ylabel('Sales')
plt.title('TV Ads Expenditure Vs. Sales')
plt.legend()
plt.show()

## Step 2: Model Definition

Now, we'll define our linear regression model as a PyTorch module.  This involves creating a class that inherits from `nn.Module` and implementing the `__init__` and `forward` methods.

Our linear model will have the form:

$$y = w^T x + b$$

where:
*   $y$ is the predicted output (Sales).
*   $x$ is the input feature vector (TV, Radio, Newspaper).
*   $w$ is the weight vector (a single tensor, not three separate weights).
*   $b$ is the bias term (a single tensor).
We do not use the ready to use nn.Linear() layer and want to write it ourselves.

In [ ]:
import torch.nn as nn

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weight = nn.Parameter(torch.rand(3))
        self.bias = nn.Parameter(torch.rand(1))

    def forward(self, x):
        y_pre_Forward = torch.matmul(x, self.weight) + self.bias
        return y_pre_Forward

## Step 3: Model Training

In this step, we'll train our linear regression model using the training data.  We'll use the following process:

1.  **Initialization:** Create an instance of our `LinearRegressionModel`.
2.  **Optimizer:** Choose an appropriate optimizer (e.g., Adam, SGD, RMSprop) and set a learning rate. We will be using Adam in this notebook. 
3.  **Loss Function:** Define a loss function to measure the difference between the model's predictions and the actual target values.  We'll use the Mean Absolute Error (MAE) loss:

    $$loss = \frac{1}{N} \sum_{i=1}^{N} |f_i - y_i|$$

    where $f_i$ is the model's output and $N$ is the number of training samples.
4.  **Training Loop:** Iterate over the training data for a specified number of epochs/iterations:
    *   Put the model into training mode
    *   Perform a forward pass through the model to get predictions.
    *   Calculate the loss.
    *   Perform backpropagation to compute gradients.
    *   Update the model's parameters using the optimizer.
    *   Clear the gradients before the next iteration (`optimizer.zero_grad()`).
    *   Put the model into evaluation mode
    *   Evaluate the loss on testing data
    *   Track training and testing losses.

In [ ]:
import torch.optim as optim

# Create an instance of the model
model = LinearRegressionModel()

# Create an optimizer with a workable learning rate
optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss()

training_losses = []
testing_losses = []

n_itr = 10000

for i in range(n_itr):
    # Train
    model.train()  # change model to train mode

    # calculate the output of the model
    y_pred = model(x_train)
    # calculate loss
    Difference = torch.abs(y_pred - y_train)
    loss = Difference.mean()

    training_losses.append(loss.item())

    # backward propagation (clear old gradients, calculate gradients, update parameters)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Evaluate, calculate the testing loss
    model.eval()  # change model to eval mode

    y_test_pred = model(x_test)
    test_Diff = torch.abs(y_test_pred - y_test)
    test_loss = test_Diff.mean()

    testing_losses.append(test_loss.item())

    # print intermediate losses for certain iterations
    if i % 100 == 1:
        print('Iteration: %04d | Training loss: %f | Testing loss: %f' %
              (i, loss.data, test_loss.data))

## Step 4: Visualization

Finally, we'll visualize the results of our training:

1.  **Loss Curves:** Plot the training and testing loss curves over the iterations. This helps us monitor the training process and check for overfitting (when the training loss decreases significantly while the testing loss plateaus or increases).
2.  **Data and Model Plot:** Plot the training data, testing data, and the fitted regression line (using the learned weights and bias) on the same plot, focusing on the 'TV' feature for visualization.

In [ ]:
# plot the training losses and testing losses
plt.figure(figsize=(10, 5))
iterations = range(n_itr)
plt.plot(iterations, training_losses, label='Train Loss', color='green')
plt.plot(iterations, testing_losses, label='Test Loss', color='red')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Extract the slope (weight) and intercept (bias) from the model
slope = model.weight[0].item()
intercept = model.bias.item()

plt.figure(figsize=(10, 6))
# Plot training data
plt.scatter(x_train[:, 0], y_train, color='green', label='Train data(%80)', s=10)
# Plot testing data
plt.scatter(x_test[:, 0], y_test, color='red', label='Test data(%20)', s=20)
# Plot the regression line
x_vals = np.linspace(x_train[:, 0].min(), x_train[:, 0].max(), 100)
y_vals = slope * x_vals + intercept
plt.plot(x_vals, y_vals, color='blue', label='Regression Line')
plt.xlabel('TV Ads')
plt.ylabel('Sales')
plt.title('TV Ads Expenditure Vs. Sales')
plt.legend()
plt.show()